In [1]:
from io import open
import os
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
SOS_token = 0
EOS_token = 1

class LanguageVocabulary(object):
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
class Preprocessing:

    DATA_DIR = '/content/drive/My Drive/Colab Notebooks/translation_data/'

    def __init__(self, lang, reverse=False, max_length_seq = 15):
        self.lang = lang
        self.source_path = os.path.join(self.DATA_DIR, lang + '.txt')
        self.target_path = os.path.join(self.DATA_DIR, 'eng_' + lang + '.txt')
        self.max_length = max_length_seq
        self.reverse = reverse

    @staticmethod
    def unicode_to_ascii(s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

    def normalize_string(self, s):
        s = self.unicode_to_ascii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Zа-яА-Яє-їЄ-Ї.!?]+", r" ", s)
        # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        return s

    def prepare_file(self, source_path, target_path):
        with open(source_path, encoding='utf-8') as file1, open(target_path, 'w', encoding='utf-8') as file2:
            for line in file1:
                file2.write(line.strip().split('\t')[0] + '\t' + line.strip().split('\t')[1] + '\n')

    def prepare_data(self, lang1, lang2, reverse=False):
        input_lang, output_lang, pairs = self.read_languages(lang1, lang2, reverse)
        print("Read %s sentence pairs" % len(pairs))
        pairs = self.filter_pairs(pairs)
        print("Trimmed to %s sentence pairs" % len(pairs))
        print("Counting words...")
        for pair in pairs:
            input_lang.add_sentence(pair[0])
            output_lang.add_sentence(pair[1])
        print("Counted words:")
        print(input_lang.name, input_lang.n_words)
        print(output_lang.name, output_lang.n_words)
        return input_lang, output_lang, pairs

    def read_languages(self, lang1, lang2, reverse=False):
        print("Reading lines...")
        path = os.path.join(self.DATA_DIR, '%s-%s.txt' % (lang1, lang2))
        lines = open(path, encoding='utf-8').read().strip().split('\n')
        pairs = [[self.normalize_string(s) for s in l.split('\t')] for l in lines]
        if reverse:
            pairs = [list(reversed(p)) for p in pairs]
            input_lang = LanguageVocabulary(lang2)
            output_lang = LanguageVocabulary(lang1)
        else:
            input_lang = LanguageVocabulary(lang1)
            output_lang = LanguageVocabulary(lang2)
        return input_lang, output_lang, pairs

    def get_data(self):
        self.prepare_file(self.source_path, self.target_path)
        input_lang, output_lang, pairs = self.prepare_data('eng', self.lang, self.reverse)
        return input_lang, output_lang, pairs

    @staticmethod
    def filter_pair(p, max_length):
        return len(p[0].split(' ')) < max_length and len(p[1].split(' ')) < max_length

    def filter_pairs(self, pairs):
        return [pair for pair in pairs if self.filter_pair(pair, self.max_length)]

MAX_LENGTH = 15
input_lang, output_lang, pairs = Preprocessing(lang='ukr', max_length_seq=MAX_LENGTH).get_data()
print(random.choice(pairs))

Reading lines...
Read 156173 sentence pairs
Trimmed to 155413 sentence pairs
Counting words...
Counted words:
eng 10014
ukr 30331
['tom is hiding behind the couch .', 'том ховається за диваном .']


In [4]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [5]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [6]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

In [7]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

In [8]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)

In [9]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [10]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

In [11]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [13]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters(encoder1, decoder1, 150000, print_every=5000)

1m 26s (- eta: 41m 39s) (5000 3%) 5.0398
2m 44s (- eta: 38m 23s) (10000 6%) 4.5353
4m 3s (- eta: 36m 35s) (15000 10%) 4.2502
5m 23s (- eta: 35m 1s) (20000 13%) 4.0189
6m 42s (- eta: 33m 34s) (25000 16%) 3.8582
8m 2s (- eta: 32m 8s) (30000 20%) 3.7093
9m 21s (- eta: 30m 44s) (35000 23%) 3.6226
10m 40s (- eta: 29m 22s) (40000 26%) 3.5143
12m 0s (- eta: 28m 0s) (45000 30%) 3.4167
13m 19s (- eta: 26m 39s) (50000 33%) 3.3383
14m 39s (- eta: 25m 18s) (55000 36%) 3.2709
15m 59s (- eta: 23m 58s) (60000 40%) 3.1636
17m 18s (- eta: 22m 38s) (65000 43%) 3.1525
18m 38s (- eta: 21m 18s) (70000 46%) 3.1129
19m 58s (- eta: 19m 58s) (75000 50%) 3.0359
21m 17s (- eta: 18m 38s) (80000 53%) 2.9818
22m 37s (- eta: 17m 18s) (85000 56%) 2.9540
23m 57s (- eta: 15m 58s) (90000 60%) 2.8836
25m 16s (- eta: 14m 38s) (95000 63%) 2.9086
26m 36s (- eta: 13m 18s) (100000 66%) 2.7977
27m 56s (- eta: 11m 58s) (105000 70%) 2.7822
29m 16s (- eta: 10m 38s) (110000 73%) 2.7341
30m 36s (- eta: 9m 18s) (115000 76%) 2.7067
3

In [14]:
evaluateRandomly(encoder1, decoder1)

> what happened last month ?
= що сталося минулого місяця ?
< що трапилося сталося ? <EOS>

> it didn t work .
= це не спрацювало .
< це не працює . <EOS>

> did tom say where he wanted to go ?
= том сказав куди він хоче піти ?
< том сказав куди куди пішов ? <EOS>

> tom will buy a new bicycle .
= том купить новии велосипед .
< том купив новии новии . <EOS>

> what s up dude ?
= як справи друже ?
< що у мене ? ? <EOS>

> sell the diamonds .
= продаи діаманти .
< кішка пістолет . <EOS>

> i like ponies .
= я люблю поні .
< я мені . . <EOS>

> please call the police .
= викличте поліцію будь ласка .
< будь ласка ласка . <EOS>

> i don t think that there will be any problems .
= не думаю що будуть які небудь проблеми .
< не думаю що що є що я маю є <EOS>

> i don t believe such things exist .
= я не вірю що такі речі існують .
< я не думаю що що у . . <EOS>



In [15]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_enc(path):
    model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

def load_dec(path):
    model = DecoderRNN(hidden_size, output_lang.n_words).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

save_model(encoder1, 'encoder_ukr_eng_v3.pt')
save_model(decoder1, 'decoder_ukr_eng_v3.pt')

In [16]:
# encoder2 = load_enc('encoder_bel_eng.pt')
# decoder2 = load_dec('decoder_bel_eng.pt')
#
# evaluateRandomly(encoder2, decoder2)